In [1]:
# =============================================================================
# =============================================================================
# # 문제 15 유형(Dataset_05_Mart_POS.csv /  이용)
#
# =============================================================================
# Dataset_05_Mart_POS.csv 
# 구분자 : comma(“,”), 20488 Rows, 3 Columns, UTF-8 인코딩
# =============================================================================
#
# 원룸촌에 위치한 A마트는 데이터 분석을 통해 보다 체계적인 재고관리와
# 운영을 하고자 한다. 이를 위해 다음의 두 데이터 세트를 준비하였다.
#
# 컬 럼 / 정 의 / Type
# Member_number / 고객 고유 번호 / Double
# Date / 구매일 / String
# itemDescription / 상품명 / String

# =============================================================================
# Dataset_05_item_list.csv 
# 구분자 : comma(“,”), 167 Rows, 4 Columns, UTF-8 인코
# =============================================================================
#
# 컬 럼 / 정 의 / Type
# prod_id / 상품 고유 번호 / Double
# prod_nm / 상품명 / String
# alcohol / 주류 상품 여부(1 : 주류) / Integer
# frozen / 냉동 상품 여부(1 : 냉동) / Integer
# =============================================================================
# =============================================================================

In [2]:
import pandas as pd

pos = pd.read_csv('Dataset/Dataset_05_Mart_POS.csv')
li = pd.read_csv('Dataset/Dataset_05_item_list.csv')

In [5]:
#%%

# =============================================================================
# 1.(Dataset_05_Mart_POS.csv를 활용하여) 가장 많은 제품이 팔린 날짜에 가장 많이 팔린
# 제품의 판매 개수는? (답안 예시) 1
# =============================================================================

In [6]:
pos['Date'].value_counts().nlargest(1)

# 정답 : 2015-01-21  96개

2015-01-21    96
Name: Date, dtype: int64

In [7]:
#%%

# =============================================================================
# 2. (Dataset_05_Mart_POS.csv, Dataset_05_item_list.csv를 활용하여) 고객이 주류 제품을
# 구매하는 요일이 다른 요일에 비해 금요일과 토요일이 많을 것이라는 가설을 세웠다. 
# 이를 확인하기 위해 금요일과 토요일의 일별 주류제품 구매 제품 수 평균과 다른
# 요일의 일별 주류제품 구매 제품 수 평균이 서로 다른지 비교하기 위해 독립 2표본
# t검정을 실시하시오. 
# 해당 검정의 p-value를 기술하시오.
# - 1분기(1월 ~ 3월) 데이터만 사용하여 분석을 실시하시오.
# - 등분산 가정을 만족하지 않는다는 조건 하에 분석을 실시하시오.
# - p-value는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 0.12
# =============================================================================

In [25]:
# 1. 데이터셋 결합
# 2. 요일
# 3. 1분기 데이터 필터링(월 단위 추출)

q2 = pos.copy()

In [26]:
# pd.to_datetime(q2.Date).dt.year
# pd.to_datetime(q2.Date).dt.month
# pd.to_datetime(q2.Date).dt.day_name(locale='ko_kr')

q2['month'] = pd.to_datetime(q2.Date).dt.month

In [27]:
q2['day'] = pd.to_datetime(q2.Date).dt.day_name()

In [28]:
q2_merge = pd.merge(q2, li, 
                    left_on='itemDescription',
                    right_on='prod_nm',
                    how = 'left')

In [30]:
q2_merge['week']=0
q2_merge.loc[q2_merge.day.isin(['Friday','Saturday']),'week']=1

In [31]:
q2_2 = q2_merge[q2_merge.month.isin([1,2,3])]

In [32]:
q2_tab = pd.pivot_table(q2_2, index='Date', columns='week',
                       values='alcohol',
                       aggfunc='sum')

In [34]:
from scipy.stats import ttest_ind

ttest_ind(q2_tab[1].dropna(), q2_tab[0].dropna(),equal_var=False)

## statistic=-2.1325016442249423, pvalue=0.035750974474819806
## 정답 : 0.04(등분산) 0.02(이분산)

Ttest_indResult(statistic=-2.335264239960428, pvalue=0.023062611047582393)

In [8]:
#%%

# =============================================================================
# 3.(Dataset_05_Mart_POS.csv를 활용하여) 1년 동안 가장 많이 판매된 10개 상품을 주력
# 상품으로 설정하고 특정 요일에 프로모션을 진행할지 말지 결정하고자 한다. 먼저
# 요일을 선정하기 전에 일원 분산 분석을 통하여 요일별 주력 상품의 판매 개수의
# 평균이 유의미하게 차이가 나는지 알아보고자 한다. 이와 관련하여 일원 분산 분석을
# 실시하고 p-value를 기술하시오.
# - p-value는 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 0.12
# 
# (참고)
# from statsmodels.formula.api import ols
# from statsmodels.stats.anova import anova_lm
# =============================================================================

In [35]:
top10 = pos['itemDescription'].value_counts().nlargest(10).index

In [37]:
q3 = q2[q2['itemDescription'].isin(top10)]

In [38]:
q3_tab = pd.pivot_table(data=q3,
                        index=['Date','day'],
                        values='itemDescription',
                       aggfunc='count').reset_index()

In [40]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [44]:
ols1 = ols('itemDescription~day', q3_tab).fit()
q3_out = anova_lm(ols1)
q3_out['PR(>F)']

# 정답 : 0.52

day         0.518128
Residual         NaN
Name: PR(>F), dtype: float64